# Workchains and high-throughput study

Workchains are key-turn solutions for a particular problem. They are essential part of AiiDA package. The main task of a workchain is to automatically perform a certain task keeping the history of all performed actions.
In this part of the tutorial you will explore and learn how to work with AiiDA-Fleur workchains.

Wokchains behave themselves similarly to a calculation: basically it can be treated as a black box that uses user's input and produces some output. For example, the SCF workchain is an engine that generates self-consistent charge denstity and other parameters for a given structure.

The other important this is that workchains can be used in a higer-level workchains.

## Workchain hierarchy 

That is the hierarchy of all implemented workchains in AiiDA-Fleur v1.0.0. Black arrows mean the use of a workchain by another one. For instance, the Structure Optimization WorkChain uses the SCF workchain inside it.
It is clealy seen that every higher-level workchain uses the SCF one. The SCF workchain itself uses FleurCalculation.

<img src="files/images/workchains.png" width="600">

In next section we will cover the SCF workchain.

## SCF WorkChain 

The SCF workchain is responsible for converging charge density for a given structure.

### Inputs of the SCF WorkChain

|name|type|describtion| required |
|:---:|:---:|:---------:|:---:|
|fleur | Code | Fleur code | yes |
|inpgen | Code | Inpgen code| no |
|wf_parameters | Dict | Settings of the workchain| no |
|structure | StructureData | Structure data node| no |
|calc_parameters | Dict | FLAPW parameters, used by inpgen| no |
|fleurinp | FleurinpData | FLEUR input files| no |
|remote_data | RemoteData | Remote folder of the previous calculation| no |
|options | Dict | AiiDA options (computational resourses)| no |
|settings | Dict | special settings for Fleur calculation| no |

Only Fleur code is required but it does not mean that one can run the SCF workchain giving only Fleur code. You **must** to follow on of the supported input configuration templates:

* fleur + fleurinp
* fleur + fleurinp + remote_data
* fleur + remote_data
* fleur + inpgen + structure

In other case the workchain will thow an error or more dangerously, perform an unexpected task.

In this tutorial we will cover only the last input configuration: we need to specify Fleur and inpgen codes and a structure. We also want to control workchain behaviour and do not use defaults for computational resources.

In [1]:
import aiida

ImportError: No module named aiida